In [1]:
import open3d as o3d
import numpy as np
import math
import copy

In [2]:
def load_point_cloud(path):
    """
    Description
    -----------
        read point cloud

    Parameters
    ----------
        path: str

    Returns
    -------
        o3d.geometry.PointCloud
    """
    cloud = o3d.io.read_point_cloud(path)
    return cloud


In [3]:
from pyquaternion import Quaternion


def matrix2rotation(matrix):
    return matrix[:3, :3]


def matrix2qua(matrix):
    qua = Quaternion(matrix=matrix2rotation(matrix))
    return qua


def matrix2trans(matrix):
    return np.array(matrix[:3, 3:].transpose())[0]


def matrix2pose(matrix):
    qua = matrix2qua(matrix)
    translation = matrix2trans(matrix)
    return np.hstack((translation, qua.elements))


def quatrans2matrix(qua, trans):
    rotaion_matrix = qua.rotation_matrix
    trans_matrix = np.array([trans]).transpose()
    tf_m = np.hstack((rotaion_matrix, trans_matrix))
    m = np.mat([[0, 0, 0, 1]])
    tf_m = np.vstack((tf_m, m))

    return tf_m


def pose2matrix(pose):
    w, x, y, z = pose[3:]
    qua = Quaternion(w=w, x=x, y=y, z=z)
    trans = pose[:3]
    return quatrans2matrix(qua, trans)


def translation2matrix(trans):
    qua = Quaternion(w=1, x=0, y=0, z=0)
    return quatrans2matrix(qua, trans)


def qua2matrix(qua):
    trans = [0, 0, 0]
    return quatrans2matrix(qua, trans)

In [4]:
help(o3d.visualization.Visualizer.add_geometry)

Help on instancemethod in module open3d.open3d.visualization:

add_geometry(...)
    add_geometry(self, geometry, reset_bounding_box=True)
    
    Function to add geometry to the scene and create corresponding shaders
    
    Args:
        geometry (open3d.geometry.Geometry): The ``Geometry`` object.
        reset_bounding_box (bool, optional, default=True): Set to ``False`` to keep current viewpoint
    
    Returns:
        bool



In [5]:
def get_picked_indices(pcd):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(window_name="Resgistration: Pick Corresponding Points")
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    return vis.get_picked_points()


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    
    world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(1)

    o3d.visualization.draw_geometries(
        [world_frame, source_temp, target], window_name="Registration Visualization"
    )
    return source_temp + target


# def icp(source, target, icp_threshold):
#     print('Please press Shift and click the feature points in the same order, press Q to finish')
#     picked_id_source = get_picked_indices(source)
#     print('picked_id_source: ', picked_id_source)
#     return picked_id
        
#     while True:
        
#         print('Please press Shift and click the feature points in the same order, press Q to finish')
#         picked_id_source = get_picked_indices(source)
#         print('picked_id_source: ', picked_id_source)
        
#         print('Please press Shift and click the feature points, press Q to finish')
#         picked_id_target = get_picked_indices(target)
#         print('picked_id_target: ', picked_id_target)

#         if len(picked_id_source) != len(picked_id_target):
#             print("The number of corresponding points are not the same, please re-pick")
#         else:
#             break

#     corr = np.zeros((len(picked_id_source), 2))
#     corr[:, 0] = picked_id_source
#     corr[:, 1] = picked_id_target

#     p2p = o3d.registration.TransformationEstimationPointToPoint()
#     trans_init = p2p.compute_transformation(
#         source, target, o3d.utility.Vector2iVector(corr)
#     )

#     reg_p2p = o3d.registration.registration_icp(
#         source,
#         target,
#         icp_threshold,
#         trans_init,
#         o3d.registration.TransformationEstimationPointToPoint(),
#     )

#     transform = reg_p2p.transformation
#     print("Transformation Matrix: \n{}\n".format(transform))

#     final_cloud = draw_registration_result(
#         source, target, transform)
    
#     return transform

In [6]:
def load_mesh_model(path):
    """
    Description
    -----------
        read mesh model

    Parameters
    ----------
        path: str

    Returns
    -------
        o3d.geometry.TriangleMesh
    """
    mesh = o3d.io.read_triangle_mesh(path)
    return mesh

def sample_mesh_to_cloud(mesh, sample_number=100000):
    """
    Description
    -----------
        process the cabinet mesh model, sample it as point cloud

    Parameters
    ----------
        mesh: o3d.geometry.TriangleMesh
        sample_number: int

    Returns
    -------
        o3d.geometry.PointCloud
    """
    mesh.compute_vertex_normals()
    pcd = mesh.sample_points_uniformly(number_of_points=sample_number)
    pcd.paint_uniform_color([1, 0.706, 0])  # yellow
    return pcd

In [33]:
path = "/home/bot/dev/projects_data/potala/potala_tote.STL"
# path = "/home/bot/dev/dr_vision_lib/modules/vision_calibration/python/example/example_data/model/shelves_3.stl"
mesh = load_mesh_model(path)
target = sample_mesh_to_cloud(mesh, 500000)
target.scale(0.001, center=False)

target.transform(pose2matrix([0,0,0,0.707,0.707,0,0]).A)

# add sign on front
add_sign = True
if add_sign:
    sign_on_front = o3d.geometry.TriangleMesh.create_box(width=0.01, height=0.01, depth=0.01)
    sign_cloud = sample_mesh_to_cloud(sign_on_front, 1000)
    sign_cloud.transform(pose2matrix([-0.5, 0, 0, 1, 0, 0, 0]).A)
    target =  target + sign_cloud


# target.transform(pose2matrix([2.98, 0.78, 0.07, 1, 0, 0, 0]).A)

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(1)


o3d.visualization.draw_geometries([world_frame, target])

In [70]:
# target1 = load_point_cloud("/home/bot/dev/projects_data/kandovan/cur/tfed_whole_model.pcd")
# target1.transform(pose2matrix([-0.32699, 1.63399, 0.47999 - 0.05, 1.0, -0.00208, -0.00061, -0.00204]).A)

target2 = load_point_cloud("/home/bot/dev/projects_data/kandovan/cur/tfed_whole_model.pcd")
target2.transform(pose2matrix([-0.34432, 2.21692, 0.02316 - 0.05, 0.99999, 0.00350, -0.00059, 0.00312]).A)

target3 = load_point_cloud("/home/bot/dev/projects_data/kandovan/cur/tfed_whole_model.pcd")
target3.transform(pose2matrix([0.32633, 1.62217, 0.02579 - 0.05, 0.99994, -0.00452, -0.00653, 0.00771]).A)

target4 = load_point_cloud("/home/bot/dev/projects_data/kandovan/cur/tfed_whole_model.pcd")
target4.transform(pose2matrix([0.31851, 2.22215, -0.05292 - 0.05, 0.99996, 0.00366, -0.00501, 0.00609]).A)

# target =  target1 + target2 + target3 + target4
target = target2 + target3 + target4

sign_on_front = o3d.geometry.TriangleMesh.create_box(width=0.01, height=0.01, depth=0.01)
sign_cloud = sample_mesh_to_cloud(sign_on_front, 1000)
sign_cloud.transform(pose2matrix([0, 1, 0, 1, 0, 0, 0]).A)
target =  target + sign_cloud

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(1)
o3d.visualization.draw_geometries([world_frame,target])

In [57]:
source = load_point_cloud("/home/bot/dev/projects_data/kandovan/2023-07-14_162036_510311716_+0800_cycle_4_kandovan_depth_camera.pcd")
# target = load_point_cloud("/home/bot/dev/projects_data/gobekli/bottom"_camera_data/1.2m/bottom_left/container_3_percipio_cloud_0.pcd")
# source.scale(0.001, center=False)

center_point = np.array([0,0,2.15])
min_bound = center_point - np.asarray([2, 2, 1.5])
max_bound = center_point + np.asarray([2, 2, 0.3])

cropbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)

source = source.crop(cropbox)
# target = target.crop(cropbox)

o3d.visualization.draw_geometries([world_frame, source])

In [26]:
source = load_point_cloud("/home/bot/dev/projects_data/gobekli/202308161248_calibration/2023-08-16_124514_769383667_+0100_cycle_56_percipio_tof_left.pcd")
target = load_point_cloud("/home/bot/dev/projects_data/gobekli/202308161248_calibration/2023-08-16_124514_970102406_+0100_cycle_56_percipio_tof_right.pcd")

In [7]:
source = load_point_cloud("/home/bot/dev/projects_data/kandovan/onsite/data/20231221_r2_right_color/2023-12-21_134046_398038789_+0800_cycle_2_kandovan_right_camera.pcd")
target = load_point_cloud("/home/bot/dev/projects_data/kandovan/onsite/data/20231221_r2_right_color/2023-12-21_134538_437162877_+0800_cycle_3_kandovan_right_camera.pcd")

In [8]:
source_ids = get_picked_indices(source)
# source_ids = [36547, 3639, 1904, 32751, 96884, 57861, 51743, 93677, 43385, 9808, 5607, 39270]
print(len(source_ids))
print(source_ids)

4
[118265, 113796, 19526, 111044]


In [9]:
target_ids = get_picked_indices(target)
print(len(target_ids))
print(target_ids)

4
[118264, 112516, 20806, 110404]


In [10]:
corr = np.zeros((len(source_ids), 2))
corr[:, 0] = source_ids
corr[:, 1] = target_ids

p2p = o3d.registration.TransformationEstimationPointToPoint()
trans_init = p2p.compute_transformation(
    source, target, o3d.utility.Vector2iVector(corr)
)
print(trans_init)

[[ 9.99987881e-01  4.92282210e-03  6.50876567e-05 -5.08492599e-04]
 [-4.92291503e-03  9.99986736e-01  1.51438071e-03 -3.47590264e-04]
 [-5.76317665e-05 -1.51468278e-03  9.99998851e-01 -6.01671998e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
draw_registration_result(source, target, trans_init)

geometry::PointCloud with 614400 points.

In [12]:
pose = matrix2pose(trans_init)
print(pose)

[-5.08492599e-04 -3.47590264e-04 -6.01671998e-03  9.99996683e-01
 -7.57268385e-04  3.06799575e-05 -2.46144244e-03]


In [13]:
reg_p2p = o3d.registration.registration_icp(
    source,
    target,
    0.008,
    trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
)

transform = reg_p2p.transformation
print("Transformation Matrix: \n{}\n".format(transform))

Transformation Matrix: 
[[ 9.99999996e-01 -6.82482665e-05  5.11866803e-05 -2.82088166e-04]
 [ 6.82501826e-05  9.99999997e-01 -3.74326677e-05 -4.97170563e-05]
 [-5.11841254e-05  3.74361611e-05  9.99999998e-01 -1.18076487e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [14]:
final_cloud = draw_registration_result(source, target, transform)

In [43]:
pose = matrix2pose(transform)
print(pose)

[-5.14432480e-02  1.66433929e+00  1.90797654e-01  9.93973686e-01
  1.03236776e-01  4.31258554e-04  3.68550377e-02]
